RAW (MOABB) to CSV

This code convert the data sets from RAW format to CSV format using MOABB.

It has been specifically conceived for BCI data.

This script is for AlexMI



In [22]:
import numpy as np
import pandas as pd
import moabb.datasets

In [25]:
#Load Database
m_dataset = moabb.datasets.AlexMI()
m_data = m_dataset.get_data()

In [3]:
#See all canal names (EEG, misc, stim...)
raw = m_data[1]['0']['0']
print("Canal list :", raw.ch_names)

Canal list : ['Fpz', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'Stim']


In [4]:
#Know what index is stim channel (we will need it later for the CSV to NY conversion)
stim_channel_name = 'Stim'
stim_idx = raw.ch_names.index(stim_channel_name)
print(f"Canal index {stim_channel_name} is : {stim_idx}")


Canal index Stim is : 16


In [5]:
#count stim data unique values (1 non target, 2 = target with a ratio needed of 5 to 1)
stim_data = raw.get_data(picks=stim_idx)
print(stim_data.shape)
unique_vals, counts = np.unique(stim_data, return_counts=True)

for val, count in zip(unique_vals, counts):
    print(f"Value : {val}, Occurences count : {count}")

(1, 254785)
Value : 0.0, Occurences count : 131905
Value : 1.0, Occurences count : 30720
Value : 2.0, Occurences count : 30720
Value : 3.0, Occurences count : 30720
Value : 4.0, Occurences count : 30720


In [6]:
stim_data = np.array(stim_data, copy=True).flatten()
stim_data[stim_data == 1] = 0
unique_vals, counts = np.unique(stim_data, return_counts=True)


In [7]:
# Durée d'un trial : 3 s * 512 Hz = 1536 échantillons.
# On souhaite, dès qu'on détecte un non zéro, laisser cet échantillon et mettre les 1535 suivants à 0.
trial_length = 1536

# Copie du signal pour le modifier
new_stim = stim_data.copy()

i = 0
while i < len(new_stim):
    if new_stim[i] != 0:
        # On conserve le signal à l'indice i (non zéro), et on met à 0 les 1535 échantillons suivants.
        start = i + 1
        end = min(i + trial_length, len(new_stim))
        new_stim[start:end] = 0
        # On passe à la fin du trial pour éviter de traiter à nouveau les échantillons du même trial.
        i = end
    else:
        i += 1


In [8]:

# Vérification : affichage des valeurs uniques après transformation.
unique_vals, counts = np.unique(new_stim, return_counts=True)
for val, count in zip(unique_vals, counts):
    print(f"(Après) Valeur : {val}, Occurrences : {count}")

(Après) Valeur : 0.0, Occurrences : 254725
(Après) Valeur : 2.0, Occurrences : 20
(Après) Valeur : 3.0, Occurrences : 20
(Après) Valeur : 4.0, Occurrences : 20


In [9]:
# Transpose to invert columns/lines
data = raw.get_data()
dataT = data.T

In [10]:
# creating timestamps and header
n_times, n_channels = dataT.shape
timestamps = np.arange(n_times, dtype=int)
data_with_timestamp = np.column_stack((timestamps, dataT))
header = [""] + [str(i) for i in range(n_channels)]

# Removing decimals from timestamps
df = pd.DataFrame(data_with_timestamp, columns=header)
df[""] = df[""].astype(int)
df.iloc[:, stim_idx + 1] = new_stim


In [11]:
# Test to check csv file
df.to_csv("data.csv", index=False)

In [26]:
subject_list = list(m_data.keys())

# Durée d'un trial en échantillons (3 s * 512 Hz)
trial_length = 1536

# Boucle sur tous les sujets
for subject in subject_list:
    # Charger l'enregistrement Raw pour le sujet (session '0', run '0')
    raw = m_data[subject]['0']['0']
    
    # Extraction de toutes les données EEG
    data = raw.get_data()         # shape: (n_channels, n_times)
    dataT = data.T                # shape: (n_times, n_channels)
    n_times, n_channels = dataT.shape
    
    # Création de la colonne de timestamps
    timestamps = np.arange(n_times, dtype=int)
    datacsv = np.column_stack((timestamps, dataT))
    header = [""] + [str(i) for i in range(n_channels)]
    df = pd.DataFrame(datacsv, columns=header)
    df[""] = df[""].astype(int)
    
    # Traitement du canal stimulation pour ce sujet
    stim_data = raw.get_data(picks=stim_idx)         # shape: (1, n_times)
    stim_data = np.array(stim_data, copy=True).flatten()  # convertit en vecteur 1D mutable
    
    # Remplacer toutes les valeurs 1 par 0
    stim_data[stim_data == 1] = 0
    
    # Créer une copie pour modification
    new_stim = stim_data.copy()
    i = 0
    while i < len(new_stim):
        if new_stim[i] != 0:
            # On a trouvé le début d'un trial : on conserve la valeur à i
            start = i + 1
            end = min(i + trial_length, len(new_stim))
            new_stim[start:end] = 0  # mettre à 0 les 1535 échantillons suivants
            i = end  # passer à la fin de ce trial
        else:
            i += 1

    # Intégrer le nouveau canal stimulation dans le DataFrame
    # La colonne dans le DataFrame correspondante au canal stimulation se trouve à stim_idx+1 (après la colonne "timestamp")
    df.iloc[:, stim_idx + 1] = new_stim
    events = df.iloc[:, stim_idx + 1]
    
    # Construction du nom de fichier
    subject_str = f"{int(subject):02d}"  # on formate le numéro de sujet sur 2 chiffres
    session_str = f"{1:02d}"            # ici, session fixée à 1 (puisqu'il s'agit du run '0' dans notre cas)
    filename = f"subject_{subject_str}_session_{session_str}.csv"
    
    # Exporter le DataFrame en CSV
    df.to_csv(filename, index=False)
    print(f"Saved file : {filename}")

        # display info
    n_rh = len(events[events == 2]) 
    n_f = len(events[events == 3]) 
    rest = len(events[events == 4])
    print(f"Nombre de Right hand (2): {n_rh}")
    print(f"Nombre de feet (3): {n_f}")
    print(f"Nombre de rest (4): {rest}")

Saved file : subject_01_session_01.csv
Nombre de Right hand (2): 20
Nombre de feet (3): 20
Nombre de rest (4): 20
Saved file : subject_02_session_01.csv
Nombre de Right hand (2): 20
Nombre de feet (3): 20
Nombre de rest (4): 20
Saved file : subject_03_session_01.csv
Nombre de Right hand (2): 20
Nombre de feet (3): 20
Nombre de rest (4): 20
Saved file : subject_04_session_01.csv
Nombre de Right hand (2): 20
Nombre de feet (3): 20
Nombre de rest (4): 20
Saved file : subject_05_session_01.csv
Nombre de Right hand (2): 20
Nombre de feet (3): 20
Nombre de rest (4): 20
Saved file : subject_06_session_01.csv
Nombre de Right hand (2): 20
Nombre de feet (3): 20
Nombre de rest (4): 20
Saved file : subject_07_session_01.csv
Nombre de Right hand (2): 20
Nombre de feet (3): 20
Nombre de rest (4): 20
Saved file : subject_08_session_01.csv
Nombre de Right hand (2): 20
Nombre de feet (3): 20
Nombre de rest (4): 20
